In [116]:
import unittest
import json
import requests
import pickle
import logging



In [117]:
class Photo():
    def __init__(self, title, author, tags):
        self.title = title
        self.author = author
        self.tags = tags
    def __str__(self):
        return "({}, {}, {})".format(self.title, self.author, self.tags)

my_photo = Photo("Photo1", "Ansel Adams", ['Nature', 'Mist', 'Mountain'])

#print my_photo

In [118]:
f = open("sample_d.txt", "r")
sample_d = json.loads(f.read())
f.close()

#print sample_d['photo']['title']['_content']
#print sample_d['photo']['owner']['username']
#print sample_d['photo']['tags']['tag'][0]['_content']

In [119]:
# Your job is to fill in the __init__ method
class Photo2():
    def __init__(self, photo_d):
        self.title = (photo_d['photo']['title']['_content']).encode('utf-8')
        self.author = (photo_d['photo']['owner']['username']).encode('utf-8')
        self.tags = [(x['raw']).encode('utf-8') for x in photo_d['photo']['tags']['tag']]
    def __repr__(self):   #using the special __repr__ method for later 
        return "({}, {}, {})".format(self.title, self.author, self.tags)
    #def __getitem__(self, item):
        #return self.item

#print Photo2(sample_d), type (Photo2(sample_d))

In [120]:
flickr_key = '88c582e55511629d1c00f987981f5a6f' # paste your flickr key here, so the variable flickr_key contains a string (your flickr key!)
if not flickr_key:
    flickr_key = raw_input("Enter your flickr API key, or paste it in the .py file to avoid this prompt in the future: \n>>")


cache_fname = "cache.bin"

#loading contents in cache file to variable saved_cache
try:
    fobj = open(cache_fname, 'rb')
    saved_cache = pickle.load(fobj)
    fobj.close()
except:
    raise Exception("Cache file not found.")


def requestURL(baseurl, params = {}):
    req = requests.Request(method = 'GET', url = baseurl, params = params)
    prepped = req.prepare()
    return prepped.url

def get_with_caching(base_url, params_diction, cache_diction, cache_fname, omitted_keys = ['api_key']):
    filtered_params_diction = {}
    for k in params_diction:
        if k not in omitted_keys:
            filtered_params_diction[k] = params_diction[k]
    full_url = requestURL(base_url, filtered_params_diction)
    #print full_url
    # step 1
    if full_url in cache_diction:
        # step 2
        logging.info("retrieving cached result for " + full_url)
        return cache_diction[full_url]
        #print cache_diction[full_url]
    else:
        # step 3
        response = requests.get(base_url, params=params_diction)
        logging.info("adding cached result for " + full_url)
        # add to the cache and save it permanently
        cache_diction[full_url] = response.text
        fobj = open(cache_fname, "wb")
        pickle.dump(cache_diction, fobj)
        fobj.close()
        return response.text

In [121]:
# [PROBLEM 3]

# Get a response from flickr: data for 50 photos that are tagged with 'sunset'. 
#Store the list of dictionaries in a variable called search_result_txt. 
#See the textbook section on the flickr API, 
#and see the documentation page at https://www.flickr.com/services/api/flickr.photos.search.html

params_d = {}
params_d['method'] = 'flickr.photos.search'
params_d['api_key'] = flickr_key
params_d['format'] = 'json'
params_d['tags'] = 'sunset'
params_d['per_page'] = 50
search_result_txt = get_with_caching('https://api.flickr.com/services/rest/', params_diction= params_d, 
                                     cache_diction = saved_cache, cache_fname = cache_fname)
#print type(search_result_txt)
#print search_result_txt

In [122]:
# [PROBLEM 4] Extract and parse as json

# search_result contains a long string, 
#with the annoying extra characters that the flickr api adds at the beginning and end of its response text. 
#Strip off the extra characters and turn the remaining json-formatted text string into a python dictionary. 
#(Hint: see the flickr section of the UsingRESTAPIs chapter for a  reminder of how to do this.) 
#Save the dictionary in a variable called search_result_diction

#this is my solution solving the "unicode" issue without involking json.loads.
import ast    #converted parsed search_result_txt to a dict
search_result_diction = ast.literal_eval(search_result_txt[14:-1])

#print search_result_diction
#print type(d)

In [123]:
# [PROBLEM 5] Extract a list of photo ids from the dictionary. Use a list comprehension or a call to map in order to do this.

# REF: Read the chapter titled: "More on Accumulation: Map, Filter, Reduce, List Comprehensions, and Zip"

photo_ids_list = [x['id'].encode('utf-8') for x in search_result_diction['photos']['photo']]
#print photo_ids_list[0]
#print photo_ids_list, type(photo_ids_list), type(photo_ids_list[0])

In [201]:
# [PROBLEM 6] Get info from flickr about each photo id

## Create an instance of Photo2 for each of the photo ids.

#a) make a request to the flickr API method flickr.photos.getInfo instead of flickr.photos.search. 
#See documentation at https://www.flickr.com/services/api/flickr.photos.getInfo.html for what to pass as extra parameters.
#b) Extract a dictionary from the response for each.
#c) Pass the dictionary when constructing a new instance of Photo2
#d) Accumulate the instance into a list, called photo_instances

def get_id_info(id):
    params_d_id = {}
    params_d_id['method'] = 'flickr.photos.getInfo'
    params_d_id['api_key'] = flickr_key
    params_d_id['format'] = 'json'
    params_d_id['photo_id'] = id
    search_result_diction_id = json.loads(get_with_caching('https://api.flickr.com/services/rest/', params_diction= params_d_id, 
                                     cache_diction = saved_cache, cache_fname = cache_fname)[14:-1])
    #search_result_diction_id ={str(key):value for key,value in search_result_diction_id.items()}  
    #above is my failed attempt to convert the dictionary to the one with unicode values
    instance = Photo2(search_result_diction_id)
    return instance
photo_instances = [get_id_info(x) for x in photo_ids_list]

#print type(photo_instances)
#print photo_instances

In [185]:
# [PROBLEM 7] Accumulate frequencies of related tags.
# You started out with data about 50 different photos, including the tags that the photo owners used to describe the photos. 
#They all have the tag 'sunset', since that's the tag we searched for, 
#but some have additional tags, like 'river' and 'nature' and others. 
#Accumulate a dictionary of counts; call the dictionary counts_diction.

# REF: See the chapter titled, "Accumulating Results in Dictionaries"

# getting all tags into a list
counts_list = [x.tags for x in photo_instances]

def get_elements(counts_list):  #flatten out the nested list 'counts_list'
    counts_list_tag = []
    for i in counts_list:
        if type(i) == list:
            counts_list_tag.extend(get_elements(i))
        else:
            counts_list_tag.append(i)
    return counts_list_tag

my_list = get_elements(counts_list)

# counting the tags
counts_diction = {} 
for i in my_list:
    if i not in counts_diction:
        counts_diction[i] = 1
    else:
        counts_diction[i] += 1

#print counts_diction

In [196]:
# [PROBLEM 8] Sort the tags

# Sort all the tags in descending order based on how often they were used in the 50 photos. 
#Save the sorted list in a variable called sorted_tags.

# REF: See the chapter titled, "Sorting"

sorted_tags = sorted(counts_diction.keys(), key=lambda x: counts_diction[x], reverse=True)

#print type(sorted_tags)
#print sorted_tags

In [200]:
# [PROBLEM 9] Output five recommended tags


## Print, for the user to see, the five tags (other than the searched on tag, sunset) that were used MOST frequently!

## HINT 1: Take a slice of the sorted list.
## HINT 2: Skip the first element in the sorted list. That will be "sunset", since all the photos have that tag.

# REF: Slicing is in one of the early chapters, titled, "Sequences"

print "Below this line, the 5 most frequently used tags should print out:"

n = 0 #counter
recommended_list = sorted_tags[1:6]
for i in recommended_list:
    n += 1
    print "{}. {}".format(n, i)

print "-----------------done; output of diagnostic tests is below this line------------"

Below this line, the 5 most frequently used tags should print out:
1. Lake Geneva
2. Andrew Wilson Switzerland
3. Lausanne
4. Switzerland
5. Sunset
-----------------done; output of diagnostic tests is below this line------------
